# Monte Carlo Simulation

In [1]:
import pandas as pd

df = pd.read_csv("C:/Users/Daniel Li/Downloads/heart_2020_cleaned.csv")
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [2]:
from numpy import random
import numpy as np
import math

random.seed(420)

#Function to convert lifestyle AgeCategory into random age within the range
def cat2age(x):
    if x == "18-24":
        return random.randint(18,25,runs)
    elif x == "25-29":
        return random.randint(25,30,runs)
    elif x == "30-34":
        return random.randint(30,35,runs)
    elif x == "35-39":
        return random.randint(35,40,runs)
    elif x == "40-44":
        return random.randint(40,45,runs)
    elif x == "45-49":
        return random.randint(45,50,runs)
    elif x == "50-54":
        return random.randint(50,55,runs)
    elif x == "55-59":
        return random.randint(55,60,runs)
    elif x == "60-64":
        return random.randint(60,65,runs)
    elif x == "65-69":
        return random.randint(65,70,runs)
    elif x == "70-74":
        return random.randint(70,75,runs)
    elif x == "75-79":
        return random.randint(75,80,runs)
    elif x == "80 or older":
        return random.randint(80,111,runs)

prob_list = []
runs = 10000

#GRACE model coefficients
coefficients = np.array([0.0531,0.0087,-0.0168,0.1823,0.6931,1.4586,0.4700,0.8755]).T

#Iterate through every row of lifestyle
for row in range(len(df)):
    #Simulation of GRACE variables
    age = cat2age(df.AgeCategory[row])
    pulse = random.normal(70.59,8.36,runs)
    bpsys = random.normal(128.4,19.6,runs)
    creat_mg = random.randint(0,40,runs)/10
    killip = random.randint(1,5,runs)
    carrst = np.array([int(df.Stroke[row]=='Yes')]*runs)
    posinit = random.randint(0,2,runs)
    stchange = random.randint(0,2,runs)
    grace = np.array([age,pulse,bpsys,creat_mg,killip,carrst,posinit,stchange]).T

    #Calculation of Risk Probability
    xb = np.sum(grace*coefficients,axis=1)-7.7035
    prob = math.e**(xb)/(1+math.e**(xb))
    prob_list.append(np.mean(prob))

In [3]:
# GRACE Model Scoring
df['grace_prob'] = prob_list
df['grace_pred'] = 'No'
df.loc[df.grace_prob>0.10,'grace_pred'] = 'Yes'

from sklearn.metrics import confusion_matrix
confmat = confusion_matrix(df.HeartDisease,df.grace_pred)
print("TPR: ",confmat[1,1]/(confmat[1,1]+confmat[1,0]))
print("TNR: ",confmat[0,0]/(confmat[0,1]+confmat[0,0]))
print('Accuracy: ',sum(df['HeartDisease'] == df['grace_pred'])/len(df))
confmat

TPR:  0.4424432835275637
TNR:  0.8604208985644035
Accuracy:  0.8246439125064494


array([[251606,  40816],
       [ 15262,  12111]], dtype=int64)

In [4]:
# Lifestyle Logistic Model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

Xx = df[['BMI','Smoking','AlcoholDrinking','Stroke','PhysicalHealth','MentalHealth','DiffWalking','Sex','AgeCategory','Race','Diabetic','PhysicalActivity','GenHealth','SleepTime','Asthma','KidneyDisease','SkinCancer']]
y = df['HeartDisease']

#Simple Handling of Categorical Variables
Ordenc = OrdinalEncoder()
OHenc = OneHotEncoder(drop='first', sparse=False)
OrdVar = Ordenc.fit_transform(df[['Smoking','AlcoholDrinking','Stroke','DiffWalking','Diabetic','PhysicalActivity','Asthma','KidneyDisease','SkinCancer']])
OHVar = OHenc.fit_transform(df[['Sex','AgeCategory','Race','GenHealth']])
X = np.concatenate((OrdVar,OHVar),axis=1)

#Lifestyle Model Scoring
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=420)
clf = LogisticRegression(solver='newton-cg',random_state=420).fit(X_train, y_train)
confmat = confusion_matrix(y,clf.predict(X))
print("TPR: ",confmat[1,1]/(confmat[1,1]+confmat[1,0]))
print("TNR: ",confmat[0,0]/(confmat[0,1]+confmat[0,0]))
print('Accuracy of Train & Test: ',clf.score(X_train, y_train),clf.score(X_test, y_test))
confmat

TPR:  0.10364227523472035
TNR:  0.9919055337833679
Accuracy of Train & Test:  0.9155552775997123 0.9163526634250067


array([[290055,   2367],
       [ 24536,   2837]], dtype=int64)

In [5]:
#Combined Model by averaging probability
@np.vectorize
def log(x):
    return 1/(1+math.e**(-x))
df['pki_prob'] = log(np.sum(X*clf.coef_[0],axis=1)+clf.intercept_[0])

#Combined Model Scoring
df['grace_comb_prob'] = (df['grace_prob']+df['pki_prob'])/2 
df['grace_comb_pred'] = 'No'
df.loc[df.grace_comb_prob>0.10,'grace_comb_pred'] = 'Yes'

confmat = confusion_matrix(df.HeartDisease,df['grace_comb_pred'])
print("TPR: ",confmat[1,1]/(confmat[1,1]+confmat[1,0]))
print("TNR: ",confmat[0,0]/(confmat[0,1]+confmat[0,0]))
print("Accuracy: ",sum(df.HeartDisease==df['grace_comb_pred'])/len(df))
confmat

TPR:  0.705585796222555
TNR:  0.7923480449487385
Accuracy:  0.784921590393846


array([[231700,  60722],
       [  8059,  19314]], dtype=int64)

- False Negative Rate () is much lower in the combined model than individual models ( for GRACE and for lifestyle)
- More deadly error as it provides a false sense of security, mislead patient into not going for early treatment

In [8]:
df.to_csv('combined_model.csv',index=False)